<a href="https://colab.research.google.com/github/fritzlabs/fritz-models/blob/using-fritz-train/NileshFingernailInvestigation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/fritzlabs/fritz-models
%cd fritz-models/
!git checkout using-fritz-train
%cd image_segmentation

fatal: destination path 'fritz-models' already exists and is not an empty directory.
/content/fritz-models
Already on 'using-fritz-train'
Your branch is up to date with 'origin/using-fritz-train'.
/content/fritz-models/image_segmentation


In [2]:
!pip install pipenv
!pipenv install --system
!pip install tensorflow-gpu

Installing dependencies from Pipfile.lock (f9cebb)…
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 26/26 — 00:00:16


In [0]:
!fritz config

In [4]:
with open('examples/labels.txt', 'w') as f:
    f.write('\n'.join(['idx,label', 'None', 'Fingernail']))

!cat examples/labels.txt

idx,label
None
Fingernail

In [0]:
!git pull

!python -m image_segmentation.train \
    --data examples/fingernail_data_23feb_not_clipped.tfrecord \
    -l examples/labels.txt \
    --use-dali \
    -s 384 \
    -a 1 \
    --batch-size 15 \
    --steps-per-epoch 500 \
    -n 50000 \
    --parallel-calls 4 \
    --lr 0.0005 \
    --model-name fingernail_test \
    --output fingernail_test_384x384.h5 \
    --fine-tune-checkpoint fingernail_test_384x384.h5 \
    --upload-to-fritz

Already up to date.
Using TensorFlow backend.
2019-02-27 09:28:42,330 - __main__ - INFO - ICNet Model training Parameters
2019-02-27 09:28:42,330 - __main__ - INFO - -------------------------------
2019-02-27 09:28:42,330 - __main__ - INFO -     data=['examples/fingernail_data_23feb_not_clipped.tfrecord']
2019-02-27 09:28:42,331 - __main__ - INFO -     tfindex_files=None
2019-02-27 09:28:42,331 - __main__ - INFO -     label_filename=examples/labels.txt
2019-02-27 09:28:42,331 - __main__ - INFO -     image_size=384
2019-02-27 09:28:42,331 - __main__ - INFO -     alpha=1.0
2019-02-27 09:28:42,331 - __main__ - INFO -     augment_images=True
2019-02-27 09:28:42,331 - __main__ - INFO -     add_noise=0.0
2019-02-27 09:28:42,331 - __main__ - INFO -     use_dali=True
2019-02-27 09:28:42,331 - __main__ - INFO -     list_labels=False
2019-02-27 09:28:42,331 - __main__ - INFO -     batch_size=15
2019-02-27 09:28:42,331 - __main__ - INFO -     lr=0.0005
2019-02-27 09:28:42,331 - __main__ - INFO - 

In [0]:
from image_segmentation.image_segmentation_records import ImageSegmentationTFRecord

fingernail_records = ImageSegmentationTFRecord('examples/fingernail_data_23feb.tfrecord')
fingernail_records_new = ImageSegmentationTFRecord('examples/fingernail_data_23feb_not_clipped.tfrecord')
fingernail_records_new.filename = fingernail_records_new.filenames[0]
fingernail_records_new.write(fingernail_records)

In [0]:
from image_segmentation.evaluate import EvaluateImageSegmentationModel
results = EvaluateImageSegmentationModel('fingernail_test_384x384.h5', 'examples/fingernail_data_23feb.tfrecord')

In [0]:

from image_segmentation.utils import plot_pixel_probabilities
from sklearn.utils.class_weight import compute_class_weight
import numpy

all_classes = [0, 0]

for record in results.tfrecords.read(decode=True):
    _, counts = numpy.unique(record['mask'], return_counts=True)
    all_classes[0] += counts[0]
    all_classes[1] += counts[1]

    prediction = results.run_prediction(record)
    results.generate_prediction_image(record, prediction)
    pyplot.show()
    plot_pixel_probabilities(prediction[0], ['none', 'fingernail'])
    pyplot.show()

print(all_classes)